In [ ]:
!pip install transformers
!pip install hazm
!pip install bert-embedding mxnet-cu100

In [2]:
# !pip install numpy --upgrade --ignore-installed

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText 
!sudo pip install .
!sudo python setup.py install
%cd ..

In [4]:
import torch
from torch.nn import functional as F
import os
import pandas as pd
import numpy as np
import re
import hazm
from hazm import *
import warnings
import mxnet as mx
from bert_embedding import BertEmbedding
import fasttext.util
import sklearn
import numpy as np
from scipy import spatial
import re
import math
from google.colab import drive

warnings.filterwarnings('ignore')


In [ ]:
!pip3 install numpy --upgrade

In [6]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("HooshvareLab/bert-fa-base-uncased")

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cpu



In [8]:
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [9]:
# os.chdir('drive/My Drive/nlp')

In [10]:
SKIPGRAM_MODEL_FILE_ID = '1wPnMG9_GNUVdSgbznQziQc5nMWI3QKNz'
CBOW_MODEL_FILE_ID = '1cQP10CGV6kAwmRuESJ5RTsgHq5TveXwV'

In [11]:
!gdown --id $SKIPGRAM_MODEL_FILE_ID 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1wPnMG9_GNUVdSgbznQziQc5nMWI3QKNz
To: /content/drive/My Drive/nlp/farsi-dedup-skipgram.bin
100% 4.37G/4.37G [01:04<00:00, 67.6MB/s]


In [12]:
model_skipgram = fasttext.load_model('farsi-dedup-skipgram.bin') #Due to the large size of this file can't upload here, please download from https://github.com/taesiri/PersianWordVectors

In [13]:
excel_data = pd.read_excel('dictionary.xlsx',header=None)
data = pd.DataFrame(excel_data)
data.rename(columns = {0:'word', 1:'synset'}, inplace = True)

In [14]:
data.head()

,word,synset
0,آباء,اجداد، اسلاف، پدران، پیشینیان، نیاکان & اخلاف
1,آباجی,آبجی، اخت، باجی، خواهر، دده، شاباجی، همشیره &...
2,آباد,1 برپا، دایر، معمور 2 پررونق 3 پیشرفته، توسعه...
3,آبادان,1 آباد، برپا، پررونق، دایر، معمور & خراب
4,آبادانی,1 آبادی، عمارت، عمران 2 ترقی، توسعه، رونق 3 آ...


In [15]:
pattern1 = r' *1 *' 
pattern = r' *[0-9]+ *'
data2 = data.dropna()
refine_data = pd.DataFrame(columns=['word','syn_list'])
refine_data2 = pd.DataFrame(columns=['word','syn_list'])

for index, row in data2.iterrows():#clean dictionary and delete antonyms
    syns = re.sub(pattern,'،',re.sub(pattern1,'',row['synset']).strip()).split('&')[0].replace('\u200c',' ')
    syn_list = [s.strip() for s in syns.split('،')]
    refine_data = refine_data.append({'word':row['word'],'syn_list':syn_list}, ignore_index = True)

    refine_data2 = pd.concat([refine_data2,pd.DataFrame.from_records({'word': row['word'], 'syn_list': syn_list})])
    
refine_data.head()


,word,syn_list
0,آباء,"[اجداد, اسلاف, پدران, پیشینیان, نیاکان]"
1,آباجی,"[آبجی, اخت, باجی, خواهر, دده, شاباجی, همشیره]"
2,آباد,"[برپا, دایر, معمور, پررونق, پیشرفته, توسعه یاف..."
3,آبادان,"[آباد, برپا, پررونق, دایر, معمور]"
4,آبادانی,"[آبادی, عمارت, عمران, ترقی, توسعه, رونق, آسایش..."


In [16]:
dictionary_words = list(pd.Series(refine_data['word']))
dictionary_words = [w.strip() for w in dictionary_words]

In [17]:
def find_most_relevant_syn(index,word,sent_word_list,topN):
    src_sent_vec = model_skipgram.get_sentence_vector(' '.join(sent_word_list))
    syn_list = refine_data.loc[dictionary_words.index(word)]['syn_list']
    
    similarity = []
    
    for i in range(len(syn_list)):#for each word in synset add cosine similarity between base sentence and replaced sentence
        sent_temp = sent_word_list[:]
        sent_temp[index] = syn_list[i]
        similarity.append(1 - spatial.distance.cosine(src_sent_vec, model_skipgram.get_sentence_vector(' '.join(sent_temp))))

    topN_ind = sorted(range(len(similarity)), key=lambda i: similarity[i])[-topN:]
    topN_ind.reverse()
    topN_syn = []
    
    for i in range(topN):
        if syn_list[topN_ind[i]] != '':
            topN_syn.append(syn_list[topN_ind[i]])
    
    return topN_syn

In [23]:
 def find_equivalent_words(text,topN = 1):
    sent_word_list = word_tokenize(text) #use hazm word tokenizer
    equivalent_words_list = []
    ind = 0

    for word in sent_word_list:
        if word in dictionary_words:
            equivalent_words_list.append(find_most_relevant_syn(ind,word,sent_word_list,topN))
        else:
            equivalent_words_list.append([])
        ind += 1

    print(equivalent_words_list)



In [24]:
find_equivalent_words('به همه مردم توصیه می‌کنیم در هنگام سفر در صورت بروز خستگی و خواب آلودگی از طی ادامه مسیر خودداری کرده و پس از استراحت به مسیر ادامه دهند.',1)

[['به منظور'], ['تمام'], ['ملت'], ['پیشنهاد'], [], ['باب'], ['موقع'], ['مسافرت'], ['باب'], ['شکل'], ['ظهور'], ['کوفتگی'], [], ['قیلوله'], ['ناپاکی'], [], [], [], ['جاده'], ['پرهیز'], [], [], ['بعد'], [], ['آسودن'], ['به منظور'], ['جاده'], [], [], []]
